<a href="https://colab.research.google.com/github/LucianoRSouza/AutomateExcelReport/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-02-19 15:12:34--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   126MB/s    in 0.2s    

2023-02-19 15:12:35 (126 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [24]:
# add your code here - consider creating a new cell for each section of code
df_books.head(10)

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
5,0399135782,The Kitchen God's Wife,Amy Tan
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley
7,0671870432,PLEADING GUILTY,Scott Turow
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie


In [6]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [7]:
df_ratings.shape, df_ratings['user'].shape #all users will be unique

((1149780, 3), (1149780,))

In [8]:
df_ratings.isnull().sum() # this shows if there are any null values

user      0
isbn      0
rating    0
dtype: int64

In [9]:
df_books.isnull().sum()

isbn      0
title     0
author    1
dtype: int64

In [10]:
df_books_cleaned = df_books.loc[~df_books.author.isnull()] # here it''ll get the author's column and loc any value that is null
df_books_cleaned.isnull().sum()

isbn      0
title     0
author    0
dtype: int64

In [12]:
df_ratings.shape, df_books_cleaned.shape

((1149780, 3), (271378, 3))

In [13]:
active_users = df_ratings.user.value_counts()[df_ratings.user.value_counts()>=200].index
popular_books = df_ratings.isbn.value_counts()[df_ratings.isbn.value_counts()>=100].index

In [15]:
df_ratings_cleaned = df_ratings.loc[(df_ratings.user.isin(active_users))]
df_books_cleaned = df_books_cleaned.loc[(df_books_cleaned.isbn.isin(popular_books))]

In [16]:
df_merged = pd.merge(left=df_ratings_cleaned, right= df_books_cleaned, on= 'isbn', how= 'inner')
print(df_merged.shape)
df_merged.head()

(49517, 5)


,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner


In [17]:
df_merged.isnull().sum()

user      0
isbn      0
rating    0
title     0
author    0
dtype: int64

In [19]:
df_unique = df_merged.drop_duplicates(['title', 'user'])
df_unique.head(10)

,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
5,16795,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
6,24194,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
7,25981,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
8,26535,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
9,28204,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner


In [20]:
df_unique.shape

(49136, 5)

In [21]:
pivot = df_unique.pivot(index= 'title', columns= 'user', values= 'rating').fillna(0)
pivot = pivot.sort_index()
pivot

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
input = pivot.values
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(input)

NearestNeighbors(algorithm='brute', metric='cosine')

In [30]:
# input
book = 'The Lovely Bones: A Novel'
pivot.loc[book, :].sum()


870.0

In [32]:
# function to return recommended books - this will be tested
def get_recommends(book = "Where the Heart Is (Oprah's Book Club (Paperback))"):
  dist, suggested_books = model.kneighbors(pivot.loc[book].values.reshape(1,-1), n_neighbors=6)
  # return suggested_books[0]
  books = pivot.iloc[suggested_books[0]].index.values
  result = list(zip(books, dist[0]))
  result[0] = result[0][0]
  result[1] = sorted([[book, distance] for book, distance in result[1:]], key=lambda x: -x[1])[:4]
  recommended_books = result[:2]
  return recommended_books

get_recommends()

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8016211],
  ['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075]]]

In [33]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075]]]
You passed the challenge! 🎉🎉🎉🎉🎉
